In [72]:
import numpy as np
from random import shuffle
# import scipy.ndimage
import scipy.io as io
import pandas as pd
# import argparse

In [2]:
input_mat = io.loadmat('Indian_pines.mat')['indian_pines']
target_mat = io.loadmat('Indian_pines_gt.mat')['indian_pines_gt']

In [4]:
input_mat.shape

(145, 145, 220)

In [5]:
target_mat.shape

(145, 145)

In [6]:
PATCH_SIZE = 3
HEIGHT = input_mat.shape[0]
WIDTH = input_mat.shape[1]
BAND = input_mat.shape[2]
CLASSES = [] 
COUNT = 200
OUTPUT_CLASSES = np.max(target_mat)
print (OUTPUT_CLASSES)

16


In [11]:
input_mat = input_mat.astype(float)
input_mat -= np.min(input_mat)
input_mat /= np.max(input_mat)

In [12]:
list_labels = [2,3,5,6,8,10,11,12,14]
train_idx = [178, 178, 178, 177, 177, 178, 178, 178, 178]

In [13]:
def Patch(height_index,width_index):
    """
    Returns a mean-normalized patch, the top left corner of which 
    is at (height_index, width_index)
    
    Inputs: 
    height_index - row index of the top left corner of the image patch
    width_index - column index of the top left corner of the image patch
    
    Outputs:
    mean_normalized_patch - mean normalized patch of size (PATCH_SIZE, PATCH_SIZE) 
    whose top left corner is at (height_index, width_index)
    """
    transpose_array = input_mat
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = transpose_array[:, height_slice, width_slice]
    mean_normalized_patch = []
    for i in range(patch.shape[0]):
        mean_normalized_patch.append(patch[i] - MEAN_ARRAY[i]) 
    
    return np.array(mean_normalized_patch)

In [14]:
MEAN_ARRAY = np.ndarray(shape=(BAND,),dtype=float)
new_input_mat = []
input_mat = np.transpose(input_mat,(2,0,1))
print(input_mat.shape)
for i in range(BAND):
    MEAN_ARRAY[i] = np.mean(input_mat[i,:,:])
    new_input_mat.append(np.pad(input_mat[i,:,:],int(PATCH_SIZE/2),'constant',constant_values = 0))
    
print (np.array(new_input_mat).shape)

(220, 145, 145)
(220, 147, 147)


In [15]:
input_mat = np.array(new_input_mat)


for i in range(OUTPUT_CLASSES):
    CLASSES.append([])
count = 0
image = []
image_label = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        curr_inp = Patch(i,j)
        curr_tar = target_mat[i , j]
        if(curr_tar!=0): #Ignore patches with unknown landcover type for the central pixel
            CLASSES[curr_tar-1].append(curr_inp)
            count += 1
print (count)

10249


In [65]:
sum1 = 0
for i in CLASSES:
    print(len(i))
    sum1+=len(i)
    
print(",", sum1)

46
1428
830
237
483
730
28
478
20
972
2455
593
205
1265
386
93
, 10249


In [67]:
1428+830+483+730+478+972+2455+593+1265

9234

In [16]:
TRAIN_PATCH,TRAIN_LABELS,TEST_PATCH,TEST_LABELS,VAL_PATCH, VAL_LABELS = [],[],[],[],[],[]
FULL_TRAIN_PATCH = []
FULL_TRAIN_LABELS = []
count = 0
for i, data in enumerate(CLASSES):
    if i+1 in list_labels:
        shuffle(data)
        TRAIN_PATCH += data[:train_idx[count]]
        TRAIN_LABELS += [count]*train_idx[count]
        VAL_PATCH += data[train_idx[count]:200]
        VAL_LABELS += [count]*(200-train_idx[count])
        TEST_PATCH += data[200:]
        TEST_LABELS += [count]*(len(data) - 200)
        count += 1

FULL_TRAIN_LABELS = TRAIN_LABELS + VAL_LABELS
FULL_TRAIN_PATCH = TRAIN_PATCH + VAL_PATCH

TRAIN_LABELS = np.array(TRAIN_LABELS)
TRAIN_PATCH = np.array(TRAIN_PATCH)
TEST_PATCH = np.array(TEST_PATCH)
TEST_LABELS = np.array(TEST_LABELS)
VAL_PATCH = np.array(VAL_PATCH)
VAL_LABELS = np.array(VAL_LABELS)
FULL_TRAIN_LABELS = np.array(FULL_TRAIN_LABELS)
FULL_TRAIN_PATCH = np.array(FULL_TRAIN_PATCH)

train_idx = list(range(len(TRAIN_PATCH)))
shuffle(train_idx)
TRAIN_PATCH = TRAIN_PATCH[train_idx]
TRAIN_LABELS = TRAIN_LABELS[train_idx]
test_idx = range(len(TEST_PATCH))
TEST_PATCH = TEST_PATCH[test_idx]
TEST_LABELS = TEST_LABELS[test_idx]
val_idx = list(range(len(VAL_PATCH)))
shuffle(val_idx)
VAL_PATCH = VAL_PATCH[val_idx]
VAL_LABELS = VAL_LABELS[val_idx]
full_train_idx = shuffle(list(range(len(FULL_TRAIN_PATCH))))
FULL_TRAIN_PATCH = FULL_TRAIN_PATCH[full_train_idx]
FULL_TRAIN_LABELS = FULL_TRAIN_LABELS[full_train_idx]

In [17]:
train = {}
train["train_patch"] = TRAIN_PATCH
train["train_labels"] = TRAIN_LABELS
io.savemat("indian_pines_Train_patch_" + str(PATCH_SIZE) + ".mat", train)
print (TRAIN_PATCH.shape)


test = {}
test["test_patch"] = TEST_PATCH
test["test_labels"] = TEST_LABELS
io.savemat("indian_pines_Test_patch_" + str(PATCH_SIZE) + ".mat", test)
print (TEST_PATCH.shape)

val = {}
val["val_patch"] = VAL_PATCH
val["val_labels"] = VAL_LABELS
io.savemat("indian_pines_Val_patch_" + str(PATCH_SIZE) + ".mat", val)
print (VAL_PATCH.shape)

full_train = {}
full_train["train_patch"] = FULL_TRAIN_PATCH
full_train["train_labels"] = FULL_TRAIN_LABELS
io.savemat("indian_pines_Full_Train_patch_" + str(PATCH_SIZE) + ".mat", full_train)
print (FULL_TRAIN_LABELS.shape)

(1600, 220, 3, 3)
(7434, 220, 3, 3)
(200, 220, 3, 3)
(1, 1800)


In [20]:
print(TRAIN_LABELS.shape)
print(TEST_LABELS.shape)
print(VAL_LABELS.shape)

(1600,)
(7434,)
(200,)


In [21]:
nval = 200
nclasses = 9

test_datamat = io.loadmat('indian_pines_Test_patch_3.mat')
test_data = test_datamat['test_patch']
test_labels = test_datamat['test_labels'].transpose()
# channels = test_data.shape[2] #test_data.shape[1]

train_datamat = io.loadmat('indian_pines_Train_patch_3.mat')
train_data = train_datamat['train_patch']
train_labels = train_datamat['train_labels'].transpose()

val_data = io.loadmat("indian_pines_Val_patch_3.mat")['val_patch']
val_labels = io.loadmat("indian_pines_Val_patch_3.mat")['val_labels'].transpose()

In [23]:
train_data[0][0]

array([[ 0.02770685, -0.02189426, -0.04131847],
       [ 0.02643502,  0.00724205, -0.04235906],
       [-0.0227036 , -0.04270592,  0.02666627]])

In [27]:
train_data[0][1]

array([[-0.02639857, -0.02639857, -0.02501113],
       [-0.02616733, -0.02709229, -0.05437868],
       [-0.01125231, -0.01171479, -0.02512675]])

In [34]:
trainData = train_data.reshape(1600, 220*3*3)

In [35]:
trainData[0][10]

-0.026398570542613964

In [36]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(trainData, train_labels) 

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [42]:
preVal = neigh.predict(val_data.reshape(200, 220*3*3))
preTest = neigh.predict(test_data.reshape(7434, 220*3*3))

In [43]:
from sklearn.metrics import accuracy_score

print("Val score:", accuracy_score(preVal, val_labels))
print("Test score:", accuracy_score(preTest, test_labels))

Val score: 0.83
Test score: 0.7278719397363466


In [69]:
from sklearn.metrics import classification_report

print("Val score:")
print(classification_report(preVal, val_labels))
print("Test score:")
print(classification_report(preTest, test_labels))

Val score:
              precision    recall  f1-score   support

           0       0.73      0.62      0.67        26
           1       0.68      0.65      0.67        23
           2       1.00      0.92      0.96        24
           3       1.00      1.00      1.00        23
           4       1.00      1.00      1.00        23
           5       0.91      0.74      0.82        27
           6       0.64      0.78      0.70        18
           7       0.59      0.81      0.68        16
           8       0.91      1.00      0.95        20

    accuracy                           0.83       200
   macro avg       0.83      0.84      0.83       200
weighted avg       0.84      0.83      0.83       200

Test score:
              precision    recall  f1-score   support

           0       0.68      0.60      0.64      1385
           1       0.68      0.55      0.61       774
           2       0.96      0.59      0.73       463
           3       0.99      0.92      0.96       570
 

In [176]:
def summary(t1, t2):
    matrix = confusion_matrix(t1, t2)
    matrixDict = {}
    z = matrix.diagonal()/matrix.sum(axis=1)
    for i in range(9):
        matrixDict[str(i)] = z[i]
    matrixDict["macro avg"] = accuracy_score(t1, t2)
    matrixDict["weighted avg"] = accuracy_score(t1, t2)
    accuracydf = pd.DataFrame(list(matrixDict.items()), columns=["class", "accuracy"]) 

    report = classification_report(t1, t2, output_dict=True)
    metricsdf = pd.DataFrame(report)
    metricsdf = metricsdf.drop(columns = ["accuracy"])
    metricsdf = metricsdf.transpose()
    metricsdf = metricsdf.drop(columns = ["support"])
    metricsdf['class'] = metricsdf.index
    metricsdf = metricsdf[["class", "precision", "recall", "f1-score"]]
    metricsdf = pd.merge(metricsdf, accuracydf, on=['class'])
    return accuracydf, metricsdf

In [166]:
from sklearn.metrics import confusion_matrix
t1 = val_labels
t2 = preVal

matrix = confusion_matrix(t1, t2)
matrixDict = {}
z = matrix.diagonal()/matrix.sum(axis=1)
for i in range(9):
    matrixDict[str(i)] = z[i]
matrixDict["macro avg"] = accuracy_score(t1, t2)
matrixDict["weighted avg"] = accuracy_score(t1, t2)
accuracydf = pd.DataFrame(list(matrixDict.items()), columns=["class", "accuracy"]) 

report = classification_report(t1, t2, output_dict=True)
metricsdf = pd.DataFrame(report)
metricsdf = metricsdf.drop(columns = ["accuracy"])
metricsdf = metricsdf.transpose()
metricsdf = metricsdf.drop(columns = ["support"])
metricsdf['class'] = metricsdf.index
metricsdf = metricsdf[["class", "precision", "recall", "f1-score"]]
metricsdf = pd.merge(metricsdf, accuracydf, on=['class'])
metricsdf.round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.62,0.73,0.67,0.73
1,1,0.65,0.68,0.67,0.68
2,2,0.92,1.00,0.96,1.00
3,3,1.00,1.00,1.00,1.00
4,4,1.00,1.00,1.00,1.00
5,5,0.74,0.91,0.82,0.91
6,6,0.78,0.64,0.70,0.64
7,7,0.81,0.59,0.68,0.59
8,8,1.00,0.91,0.95,0.91
9,macro avg,0.84,0.83,0.83,0.83


In [167]:
matrix

array([[16,  2,  0,  0,  0,  2,  2,  0,  0],
       [ 2, 15,  0,  0,  0,  1,  2,  2,  0],
       [ 0,  0, 22,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, 23,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 23,  0,  0,  0,  0],
       [ 1,  1,  0,  0,  0, 20,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  2, 14,  1,  0],
       [ 4,  3,  0,  0,  0,  2,  0, 13,  0],
       [ 0,  0,  2,  0,  0,  0,  0,  0, 20]])

In [152]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(trainData, train_labels.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [153]:
preVal2 = clf.predict(val_data.reshape(200, 220*3*3))
preTest2 = clf.predict(test_data.reshape(7434, 220*3*3))

In [155]:
out = summary(preVal2, val_labels)

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [168]:
from sklearn.metrics import confusion_matrix
t1 = val_labels
t2 = preVal2

matrix = confusion_matrix(t1, t2)
matrixDict = {}
z = matrix.diagonal()/matrix.sum(axis=1)
for i in range(9):
    matrixDict[str(i)] = z[i]
matrixDict["macro avg"] = accuracy_score(t1, t2)
matrixDict["weighted avg"] = accuracy_score(t1, t2)
accuracydf = pd.DataFrame(list(matrixDict.items()), columns=["class", "accuracy"]) 

report = classification_report(t1, t2, output_dict=True)
metricsdf = pd.DataFrame(report)
metricsdf = metricsdf.drop(columns = ["accuracy"])
metricsdf = metricsdf.transpose()
metricsdf = metricsdf.drop(columns = ["support"])
metricsdf['class'] = metricsdf.index
metricsdf = metricsdf[["class", "precision", "recall", "f1-score"]]
metricsdf = pd.merge(metricsdf, accuracydf, on=['class'])
metricsdf.round(2)

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,class,precision,recall,f1-score,accuracy
0,0,0.22,0.86,0.35,0.86
1,1,1.00,0.05,0.09,0.05
2,2,0.20,0.14,0.16,0.14
3,3,0.50,0.48,0.49,0.48
4,4,0.00,0.00,0.00,0.00
5,5,0.30,0.45,0.36,0.45
6,6,0.00,0.00,0.00,0.00
7,7,0.00,0.00,0.00,0.00
8,8,0.56,1.00,0.72,1.00
9,macro avg,0.31,0.33,0.24,0.33


In [169]:
matrix

array([[19,  0,  0,  0,  0,  3,  0,  0,  0],
       [12,  1,  0,  1,  0,  7,  1,  0,  0],
       [ 0,  0,  3,  2,  0,  0,  0,  0, 17],
       [ 0,  0, 12, 11,  0,  0,  0,  0,  0],
       [14,  0,  0,  7,  0,  0,  0,  2,  0],
       [11,  0,  0,  1,  0, 10,  0,  0,  0],
       [18,  0,  0,  0,  0,  4,  0,  0,  0],
       [13,  0,  0,  0,  0,  9,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0, 22]])

In [170]:
accuracy_score(t1, t2)

0.33

In [171]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(trainData, train_labels.ravel())

/home/ajay/anaconda3/envs/baseml/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [172]:
preVal3 = rf.predict(val_data.reshape(200, 220*3*3))
preTest3 = rf.predict(test_data.reshape(7434, 220*3*3))

In [173]:
from sklearn.metrics import confusion_matrix
t1 = val_labels
t2 = preVal3

matrix = confusion_matrix(t1, t2)
matrixDict = {}
z = matrix.diagonal()/matrix.sum(axis=1)
for i in range(9):
    matrixDict[str(i)] = z[i]
matrixDict["macro avg"] = accuracy_score(t1, t2)
matrixDict["weighted avg"] = accuracy_score(t1, t2)
accuracydf = pd.DataFrame(list(matrixDict.items()), columns=["class", "accuracy"]) 

report = classification_report(t1, t2, output_dict=True)
metricsdf = pd.DataFrame(report)
metricsdf = metricsdf.drop(columns = ["accuracy"])
metricsdf = metricsdf.transpose()
metricsdf = metricsdf.drop(columns = ["support"])
metricsdf['class'] = metricsdf.index
metricsdf = metricsdf[["class", "precision", "recall", "f1-score"]]
metricsdf = pd.merge(metricsdf, accuracydf, on=['class'])
metricsdf.round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.63,0.77,0.69,0.77
1,1,0.67,0.64,0.65,0.64
2,2,0.91,0.95,0.93,0.95
3,3,0.96,1.00,0.98,1.00
4,4,1.00,1.00,1.00,1.00
5,5,0.78,0.82,0.80,0.82
6,6,0.57,0.36,0.44,0.36
7,7,0.58,0.64,0.61,0.64
8,8,1.00,0.95,0.98,0.95
9,macro avg,0.79,0.79,0.79,0.80


In [175]:
summary(t1, t2).round(2)

,class,precision,recall,f1-score,accuracy
0,0,0.63,0.77,0.69,0.77
1,1,0.67,0.64,0.65,0.64
2,2,0.91,0.95,0.93,0.95
3,3,0.96,1.00,0.98,1.00
4,4,1.00,1.00,1.00,1.00
5,5,0.78,0.82,0.80,0.82
6,6,0.57,0.36,0.44,0.36
7,7,0.58,0.64,0.61,0.64
8,8,1.00,0.95,0.98,0.95
9,macro avg,0.79,0.79,0.79,0.80


In [159]:
train_data.shape

(1600, 220, 3, 3)

In [162]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Reshape((band_size, patch_size*patch_size)))
model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(220,3,3)))
model.add(Conv2D(32, kernel_size=3, activation="relu"))
model.add(Flatten())
model.add(Dense(10, activation="softmax"))

Using TensorFlow backend.


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_2/convolution' (op: 'Conv2D') with input shapes: [?,218,1,64], [3,3,64,32].

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)